In [1]:
import numpy as np
import sys
import os
import matplotlib.pyplot as plt
import copy
import torch as tr

# import scripts
from src.es import random_opt
from src.es import evolutionary_strategy
from src.grad_opt import ADAM_opt
from src.api_helper import api_utils
from src.api_helper import env
import copyreg

device = tr.device("cuda" if tr.cuda.is_available() else "cpu")
dtype = tr.float32

##### data

In [2]:
## multi-process has problem with decorator
# def af(query): return env.rosenbrock(query); api = api_utils.multi_process_transform(af) 

@api_utils.transform
def api(query): return env.rosenbrock(query)

# @api_utils.multi_thread_transform
# def api(query): return env.rosenbrock(query)

x0, y0 = tr.rand(2).view(1, -1), tr.tensor([[-1]], dtype=dtype).view(-1, 1)  
r0 = env.rosenbrock(x0)  # initial reward on which normalisation based

x0, y0= x0.to(device), y0.to(device)  # data in GPU

print("init query:", x0); print(f"init reward: {api(x0, r0, device)}, origin reward: {r0.item():.2f}")  # x,y stay on GPU while r0 in cpu



init query: tensor([[0.2649, 0.6295]])
init reward: tensor([[-1.]]), origin reward: 31.82


##### OPT

In [3]:
# hyperparameters
T = 10  # total number of iterations

adam = ADAM_opt(); ram_opt = random_opt(); es = evolutionary_strategy()

In [18]:
%%time
es.outer_loop(T, x0, r0, api)

CPU times: user 122 ms, sys: 0 ns, total: 122 ms
Wall time: 127 ms


(tensor([[2.6491e-01, 6.2950e-01],
         [5.3031e-01, 1.2579e+00],
         [1.0612e+00, 2.5151e+00],
         [2.1233e+00, 5.0298e+00],
         [4.2476e+00, 1.0059e+01],
         [8.4940e+00, 2.0119e+01],
         [1.6987e+01, 4.0238e+01],
         [3.3973e+01, 8.0476e+01],
         [6.7945e+01, 1.6095e+02],
         [1.3589e+02, 3.2190e+02],
         [2.7178e+02, 6.4381e+02]]),
 tensor([-1.0000e+00, -7.7853e-02, -1.1123e+00, -1.8900e+00, -4.8308e-02,
         -1.0612e+02, -6.9218e+03, -1.8059e+05, -3.5094e+06, -6.1438e+07,
         -1.0268e+09]))

In [19]:
%%time
ram_opt.outer_loop(T, (-1,1), x0, r0, api, batch_size=2)

CPU times: user 9.5 ms, sys: 1.2 ms, total: 10.7 ms
Wall time: 10.8 ms


(tensor([[ 0.2649,  0.6295],
         [-0.0015,  0.6903],
         [-0.1259, -0.4915],
         [ 0.5684,  0.5299],
         [ 0.3647, -0.7769],
         [-0.4495,  0.0864],
         [-0.7185,  0.6443],
         [ 0.9188,  0.1488],
         [ 0.8105,  0.3270],
         [ 0.6440,  0.6845],
         [ 0.1147,  0.7005],
         [-0.4259,  0.3647],
         [-0.3816,  0.8107],
         [ 0.1317, -0.9143],
         [ 0.6803, -0.6395],
         [ 0.3072,  0.8241],
         [ 0.2595,  0.4973],
         [ 0.5388, -0.2813],
         [-0.5995, -0.0318],
         [-0.1350, -0.6029],
         [-0.9475,  0.1653]]),
 tensor([-1.0000, -1.5288, -0.8487, -0.1403, -2.6142, -0.1081, -0.1443, -1.5193,
         -0.3431, -0.2326, -1.5093, -0.1694, -1.4496, -2.7508, -3.8210, -1.6886,
         -0.5981, -1.0336, -0.5612, -1.2528, -1.8052]))

In [20]:
%%time
adam.outer_loop(T, x0, r0, api, env.rosenbrock_grad)

CPU times: user 11.5 ms, sys: 2.06 ms, total: 13.6 ms
Wall time: 14.7 ms


(tensor([[0.2649, 0.6295],
         [0.2659, 0.6285],
         [0.2669, 0.6275],
         [0.2679, 0.6265],
         [0.2689, 0.6255],
         [0.2699, 0.6245],
         [0.2709, 0.6235],
         [0.2719, 0.6225],
         [0.2729, 0.6215],
         [0.2739, 0.6205]]),
 tensor([-1.0000, -0.9946, -0.9892, -0.9838, -0.9784, -0.9730, -0.9676, -0.9623,
         -0.9569, -0.9516]))

In [21]:
x0

tensor([[0.2649, 0.6295]])